In [2]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re

# # Importar librerías para automatización de navegadores web con Selenium
# from selenium import webdriver  
# from selenium.webdriver.common.by import By
# from webdriver_manager.chrome import ChromeDriverManager  
# from selenium.webdriver.common.keys import Keys  
# from selenium.webdriver.support.ui import Select  
# # Importar librerías para pausar la ejecución

from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [5]:
with open('lista_pelis_2000_2006.json', 'w') as archivo:
    json.dump(lista_pelis, archivo)

NameError: name 'json' is not defined

In [ ]:
df_pelis = pd.DataFrame(lista_pelis, columns=['Tipo', 'Título', 'Año de estreno', 'Mes de estreno', 'ID'])

In [2]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"


driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

#Preguntar al usuario el producto a buscar 
pelicula = input("De que pelicula quieres los datos?")

## MODIFICAR, tiene que ser una lista de tuplas
diccionario_peliculas = {"Puntuacion_IMDB":[], "Dirección": [],"Gionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}

#Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)


sleep(5)

# Entramos a la pagina de la peli
driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-a8b15070-0.iMRIeM > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.iUyFfD > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()

sleep(5)

print(driver.find_elements("xpath", "//div[contains(@data-testid,'storyline-plot-summary')]"))

diccionario_peliculas["Nombre de la película"].append(pelicula)
diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text)
diccionario_peliculas["Dirección"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text)
diccionario_peliculas["Gionistas"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li > a").text)
diccionario_peliculas["Argumento"].append(driver.find_element("xpath", "//*[@id='__next']/main/div/section[1]/div/section/div/div[1]/section[5]/div[2]/div[1]/div/div/div").text)
diccionario_peliculas["Duración"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > ul").text)



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.57)
Stacktrace:
	GetHandleVerifier [0x00007FF6260D4C52+3505186]
	(No symbol) [0x00007FF625D00812]
	(No symbol) [0x00007FF625BB4145]
	(No symbol) [0x00007FF625B91D4C]
	(No symbol) [0x00007FF625C225F7]
	(No symbol) [0x00007FF625C37A61]
	(No symbol) [0x00007FF625C1BC33]
	(No symbol) [0x00007FF625BED618]
	(No symbol) [0x00007FF625BEE6B1]
	GetHandleVerifier [0x00007FF6261067AD+3708797]
	GetHandleVerifier [0x00007FF62615FC2D+4074493]
	GetHandleVerifier [0x00007FF626157DAF+4042111]
	GetHandleVerifier [0x00007FF625E2A0F6+708806]
	(No symbol) [0x00007FF625D0CACF]
	(No symbol) [0x00007FF625D07CD4]
	(No symbol) [0x00007FF625D07E2C]
	(No symbol) [0x00007FF625CF7964]
	BaseThreadInitThunk [0x00007FFBCA91257D+29]
	RtlUserThreadStart [0x00007FFBCC3AAA58+40]


In [55]:
diccionario_peliculas

{'Puntuacion_IMDB': ['3,7'],
 'Dirección': ['Ciro Ippolito'],
 'Gionistas': ['Ciro Ippolito'],
 'Argumento': [],
 'Duración': [],
 'Nombre de la película': ['alien 2']}

In [63]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()


sleep(5)


##CAMBIAR POR UN BUBLE QUE RECORRA UN DF
pelicula = input("De que pelicula quieres los datos?")


## MODIFICAR, tiene que ser una lista de tuplas
diccionario_peliculas = {"Rotten Tomatoes":[]}
busqueda = f"puntuacion Rotten Tomatoes de la {pelicula}"



#Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
driver.find_element("css selector",'#APjFqb').send_keys(busqueda, Keys.ENTER)


sleep(5)
try:
    diccionario_peliculas["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:nth-child(3) > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
except:
    puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
    puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
    diccionario_peliculas["Rotten Tomatoes"].append(puntuacion.group())




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#rso > div:nth-child(1) > div > div > div:nth-child(3) > div"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF63BFD7012+3522402]
	(No symbol) [0x00007FF63BBF8352]
	(No symbol) [0x00007FF63BAA5ABB]
	(No symbol) [0x00007FF63BAEBF0E]
	(No symbol) [0x00007FF63BAEC08C]
	(No symbol) [0x00007FF63BB2E437]
	(No symbol) [0x00007FF63BB0F09F]
	(No symbol) [0x00007FF63BB2BDA3]
	(No symbol) [0x00007FF63BB0EE03]
	(No symbol) [0x00007FF63BADF4D4]
	(No symbol) [0x00007FF63BAE05F1]
	GetHandleVerifier [0x00007FF63C009B9D+3730157]
	GetHandleVerifier [0x00007FF63C05F02D+4079485]
	GetHandleVerifier [0x00007FF63C0575D3+4048163]
	GetHandleVerifier [0x00007FF63BD2A649+718233]
	(No symbol) [0x00007FF63BC04A3F]
	(No symbol) 

In [62]:
print(diccionario_peliculas)

{'Rotten Tomatoes': ['80%']}
